# Cevovodi (engl. pipeline)

U ovoj sveci upoznaćemo koncept cevovoda (engl. pipeline) koji se koristi za predstavljanje sekvence koraka. Može se koristiti uključivanjem paketa `pipeline` biblioteke `scikit-learn`.

In [1]:
from sklearn import pipeline

In [2]:
from sklearn import datasets
from sklearn import linear_model
from sklearn import model_selection
from sklearn import preprocessing

Primetili smo da prilikom razvoja modela neke korake ponavljamo. Tako, na primer, vrednosti na osnovu kojih vršimo standardizaciju podataka izračunavamo na skupu za treniranje, a zatim transformišemo i skup za treniranje i skup za testiranje koristeći ove vrednosti pozivom funkcije `transform`. Slično važi i za ostale pripreme i određivanja atributa: nad skupom za treniranje se izdvoje odgovarajući atributi, a zatim se transformiše i skup za treniranje i skup za testiranje u terminima ovih atributa. `Cevovod` je mehanizam koji definiše niz koraka koje treba zajedno sprovesti, jedan za drugim. U cevovod se mogu smesiti sve bibliotečke klase koje raspolažu funkcijama `fit` i `transform`. 

Da bismo demonstrirali kako se radi sa cevovodima, učitaćemo skup koji se koristi za klasifikaciju tumora na benigne i maligne i podelićemo ga na skup za treniranje i skup za testiranje.

In [3]:
data = datasets.load_breast_cancer()

In [4]:
X = data.data
y = data.target

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.33, stratify = y, random_state = 10)

Dalje ćemo pripremiti podatke tako što ćemo ih standardizovati. 

In [6]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

Nad ovim podacima možemo obučiti klasifikacioni model po izboru. Neka to bude model logističke regresije sa regularizacionim parametrom 2 (parametar C odgovara njegovoj recipročnoj vrednosti).

In [7]:
model = linear_model.LogisticRegression(C = 1/2)

In [8]:
model.fit(X_train_transformed, y_train)

LogisticRegression(C=0.5)

Sam model evaluiramo pozivom metode `score` ili pozivom funkcije `predict` i izborom odgovarajuće mere paketa `metrics`.

In [9]:
model.score(X_test_transformed, y_test)

0.973404255319149

Kod koji sledi koristi cevovode i istovetan je sa kodom koji smo opisali. Funkcija `make_pipeline` nam omogućava da kreiramo rudimentarni cevovod koji u našem slučaju čine standardni skaler i sam model. Kada se nad cevovodom pozove funkcija `fit` ona se poziva nad svakim navedenim elementom. Poziv funkcije `score` nad cevovodom podrazumeva pozive funkcije `transform` nad koracima koji prethode poslednjem, a zatim i pozov funkcije `score` nad poslednjim korakom. Prilikom izvršavanja funkcije `transform` koriste se vrednosti koje su izračunate/dobijene prilikom poziva funkcije `fit`. Ovakav opis nam omogućava kompaktniji zapis i sprečava mogućnost previda u vidu izostavljanja nekog od koraka ili njegove primene nad pogrešnim skupom.  

In [10]:
linreg_pipeline = pipeline.make_pipeline(preprocessing.StandardScaler(), linear_model.LogisticRegression(C=1/2))

In [11]:
linreg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(C=0.5))])

In [12]:
linreg_pipeline.score(X_test, y_test)

0.973404255319149

Osim funkcije `make_pipeline` može se koristiti i klasa `Pipeline` koja nudi nešto više udobnosti u radu sa cevovodima. Ona omogućava da se prilikom kreiranja cevovoda navede imenovana lista koraka što može biti zgodno zbog pristupa pojedinačnim koracima u cilju podešavanja njihovih hiperparametara ili očitavanju dobijenih vrednosti. 

In [13]:
linreg_pipeline = pipeline.Pipeline(steps=[('scaler', preprocessing.StandardScaler()), ('linreg', linear_model.LogisticRegression())])

Metodom `set_params` možemo podesiti parametre bilo kog elementa cevovoda. Parametri se opisuju imenom elementa cevovoda za kojim se navodi dvostruka podvlaka a zatim i samo ime parametra. Ukoliko je potrebno navesti vrednosti više parametara odjednom, mogu se razdvojiti zarezom.

In [14]:
linreg_pipeline.set_params(linreg__C=2)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linreg', LogisticRegression(C=2))])

Poziv metode `fit` funkcioniše na opisani način.

In [15]:
linreg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linreg', LogisticRegression(C=2))])

Pojedinačne elemente cevovoda možemo dohvatiti korišćenjem navedenih imena. Tako se, recimo, sledećim blokom koda mogu izdvojiti naučeni koeficijenti modela logističke regresije.

In [16]:
linreg_pipeline['linreg'].coef_

array([[-8.07005546e-01, -4.66088397e-01, -7.62138451e-01,
        -8.62764726e-01, -3.22306211e-01,  5.88885736e-01,
        -1.11107982e+00, -9.75728595e-01,  4.71543320e-01,
         3.69756385e-01, -1.45928972e+00,  5.15532705e-01,
        -9.50821731e-01, -1.11160016e+00, -4.29506758e-01,
         8.24715775e-01,  4.54784673e-02,  1.39541850e-03,
        -3.02985183e-01,  8.79510794e-01, -1.29621457e+00,
        -1.49341043e+00, -1.08657756e+00, -1.20923905e+00,
        -1.33530862e+00,  2.84965566e-01, -1.03589969e+00,
        -5.01510523e-01, -8.00324199e-01, -8.14707759e-01]])

In [17]:
linreg_pipeline['linreg'].intercept_

array([-0.18591049])

Poziv metode `score`, takođe, funkcioniše na način opisan iznad.

In [18]:
linreg_pipeline.score(X_test, y_test)

0.973404255319149

Isto važi i za metod `predict`. 

In [19]:
linreg_pipeline.predict([X_test[0,:]])

array([1])

U slučajevima kada se cevovod sastoji od većeg broja koraka, zgodno je prikazati ga i grafički. To se može postići uključivanjem interaktivnog moda na nivou `scikit-learn` biblioteke.

In [20]:
from sklearn import set_config

In [21]:
set_config(display='diagram')

In [22]:
linreg_pipeline

Pipeline(steps=[('scaler', StandardScaler()),
                ('linreg', LogisticRegression(C=2))])